In [ ]:
# ✅ STEP 1: Install dependencies
!pip install -q "git+https://github.com/FlagOpen/FlagEmbedding.git"
!pip install -q nano-vectordb tiktoken transformers huggingface_hub pymupdf
from FlagEmbedding import BGEM3FlagModel
# ✅ 1. Clone PathRAG repo
!git clone https://github.com/BUPT-GAMMA/PathRAG.git
#%cd /content/PathRAG
#!pip install -e .
import fitz  # PyMuPDF

# ✅ 2. Add PathRAG to Python path
import sys
sys.path.append("/content/PathRAG")

# ✅ 3. Install required dependencies
!pip install aioboto3 aiohttp ollama

# ✅ 4. Mount Google Drive
from google.colab import drive



drive.mount('/content/drive')
pdf_path = "/content/drive/My Drive/ProstateCancer_2025.pdf"
doc = fitz.open(pdf_path)
full_text = "\n".join([page.get_text() for page in doc])

# ✅ STEP 3: Load FlagEmbedding BGE model
from FlagEmbedding import FlagModel
embedding_model = BGEM3FlagModel('BAAI/bge-m3', query_instruction_for_retrieval="", use_fp16=False)

# ✅ STEP 4: Wrap async embedding function
import asyncio

class HFEmbeddingWrapper:
    def __init__(self, model):
        self.model = model
        self.embedding_dim = 1024  # Use 1024 for BGE-M3

    async def __call__(self, texts: list[str]) -> list:
        output = await asyncio.to_thread(self.model.encode, texts)
        return output["dense_vecs"].tolist()  # ✅ Correct key and convert to list



embedding_func = HFEmbeddingWrapper(embedding_model)

# ✅ STEP 5: Setup PathRAG

from PathRAG.llm import hf_model_complete

from functools import partial
# ✅ FIXED Dummy cache setup
import os
os.makedirs("./PathRAG_cache", exist_ok=True)


from PathRAG.PathRAG import PathRAG
from PathRAG.storage import JsonKVStorage

class PathRAGPatched(PathRAG):
    def __post_init__(self):
        import os
        os.makedirs(self.working_dir, exist_ok=True)
        self.llm_response_cache = JsonKVStorage(
            namespace="llm_response_cache",
            global_config={"working_dir": self.working_dir},
            embedding_func=self.embedding_func,
        )
        super().__post_init__()  # proceed with normal setup

from PathRAG.storage import JsonKVStorage

dummy_kv = JsonKVStorage(
    namespace="dummy_kv",
    global_config={"working_dir": "./PathRAG_cache"},
    embedding_func=embedding_func,
)

# HuggingFace model for LLM completion
llm_model_name = "HuggingFaceH4/zephyr-7b-beta"
llm_func = partial(hf_model_complete, hashing_kv=dummy_kv)

# ✅ STEP 6: Instantiate PathRAG
rag = PathRAGPatched(
    working_dir="./PathRAG_cache",
    llm_model_func=llm_func,
    llm_model_name=llm_model_name,
    embedding_func=embedding_func,
    enable_llm_cache=True,
)


# ✅ STEP 7: Insert PDF content and build knowledge graph
await rag.ainsert(full_text)

# ✅ STEP 8: Query the graph
from PathRAG.base import QueryParam

query = "What are the treatment options for high-risk prostate cancer?"
param = QueryParam(mode="hybrid", top_k=5)

response = rag.query(query, param)
print("📌 Query Result:\n", response)


  Preparing metadata (setup.py) ... done
fatal: destination path 'PathRAG' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/6 [00:00<?, ?batch/s]You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Extracting entities from chunks:   0%|          | 0/192 [00:00<?, ?chunk/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

...

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [10]:
import shutil
shutil.rmtree("./PathRAG_cache", ignore_errors=True)
